# Metadata Extraction and Analysis

## Imports and dataloading

In [2]:
import numpy as np
import pandas as pd
import json
import seaborn as sns
import matplotlib.pyplot as plt
from ieeg.auth import Session
from scipy import signal as sig
import os
from os.path import join as ospj
from os.path import exists as ospe
import pathlib
from tqdm import tqdm

from utils import *

In [3]:
import sys
sys.path.append('/users/wojemann/iEEG_processing')
from pioneer import Pioneer

In [4]:
usr,passpath,datapath,prodatapath,figpath,patient_table,rid_hup,pt_list = \
load_config(ospj('/mnt/leif/littlab/users/wojemann/stim-seizures/code','config.json'),'HUP')

In [5]:
patient_table

,ptID,ieeg_ids,lf_stim,hf_stim,typical,mtle,interictal_training
13,HUP224,"[HUP224_phaseII, HUP224_CCEP]",1,0,0.0,1.0,"[HUP224_phaseII, 5915]"
14,HUP225,"[HUP225_phaseII, HUP225_CCEP]",1,0,1.0,0.0,"[HUP225_phaseII, 71207]"
15,HUP229,"[HUP229_phaseII, HUP229_CCEP]",1,0,1.0,1.0,"[HUP229_phaseII, 149146]"
16,HUP230,"[HUP230_phaseII, HUP230_CCEP]",1,0,1.0,1.0,"[HUP230_phaseII, 25350]"
17,HUP235,"[HUP235_phaseII, HUP235_CCEP]",1,1,1.0,1.0,"[HUP235_phaseII, 307651]"
18,HUP238,"[HUP238_phaseII, HUP238_CCEP]",1,0,1.0,1.0,"[HUP238_phaseII, 100011]"
19,HUP246,"[HUP246_phaseII, HUP246_CCEP]",1,0,1.0,1.0,"[HUP246_phaseII, 100000]"
20,HUP247,"[HUP247_phaseII, HUP247_CCEP]",1,0,1.0,0.0,"[HUP247_phaseII, 17590]"
21,HUP249,"[HUP249_phaseII, HUP249_CCEP]",1,0,0.0,0.0,"[HUP249_phaseII, 24112]"
22,HUP250,"[HUP250_phaseII, HUP250_CCEP]",1,0,1.0,0.0,"[HUP250_phaseII, 24841]"


## Creating annotation assignments

In [5]:
seizures_df = pd.read_csv(ospj(datapath,'stim_seizure_information - LF_seizure_annotation.csv'))
seizures_df = seizures_df[~seizures_df.Patient.isin(["HUP235","HUP238","HUP246","HUP261"])]
seizures_df = seizures_df[seizures_df['to_annotate']==1]
seizures_df['annotators'] = ""
seizures_df['approximate_onset'].fillna(seizures_df['UEO'],inplace=True)
seizures_df['approximate_onset'].fillna(seizures_df['EEC'],inplace=True)
seizures_df['approximate_onset'].fillna(seizures_df['Other_onset_description'],inplace=True)
seizures_df = seizures_df.drop(['to_annotate','Notes','source','EEC onset channels','UEO onset channels','EEC','UEO','Other_onset_description'],axis=1).reset_index(drop=True)
seizures_df.head()

,Patient,IEEGname,approximate_onset,end,stim,stim_channels,annotators
0,HUP224,HUP224_phaseII,71156.5900,71190.99,1.0,LB1-LB2,
1,HUP224,HUP224_phaseII,339143.6435,339234.20,0.0,NaN,
2,HUP224,HUP224_phaseII,491467.8046,491541.43,0.0,NaN,
3,HUP224,HUP224_phaseII,519177.9500,519258.16,0.0,NaN,
4,HUP225,HUP225_phaseII,159834.1400,159913.05,1.0,RC1-RC2,


In [6]:
# Assuming you have a DataFrame named 'seizures_df' containing seizure data
# And a list of annotators initials 2,5
np.random.seed(10)
annotators = ['CK','EC','DZ','JJ','JK']
annotation_counts = {key: 0 for key in annotators}
def calc_weights(annotation_counts):
    weights = [1/(1+value) for value in annotation_counts.values()]
    tot_weight = sum(weights)
    return [w/tot_weight for w in weights]

# Assuming 'seizures_df' contains a column 'patient_id' indicating the patient ID for each seizure
# We'll first group seizures by patient_id
grouped_seizures = seizures_df.groupby('Patient')

# Dictionary to store DataFrames for each annotator
annotator_dfs = {}

# Create Primary DF that contains all seizures from all patients with all annotators per seizure
# Iterate over each patient group
for patient_id, patient_group in grouped_seizures:
    num_seizures = len(patient_group)
    # Randomly assign 3 annotators to the patient
    weights = calc_weights(annotation_counts)
    assigned_annotators = np.random.choice(annotators, size=3, replace=False,p=weights)
    annot_str = str(assigned_annotators)
    
    annotator_list = np.repeat(annot_str,num_seizures,0)
    # if len(annotator_list.shape) < 2:
    #     annotator_list = np.expand_dims(annotator_list,0)
    
    seizures_df.iloc[seizures_df.Patient == patient_id,-1] = annotator_list
    # Repeat the annotators for each seizure in the patient group
    for annotator in assigned_annotators:
        annotation_counts[annotator] += len(patient_group)
        if annotator in annotator_dfs.keys():
            annotator_dfs[annotator] = pd.concat([annotator_dfs[annotator],patient_group])
        else:
            annotator_dfs[annotator] = patient_group
print(annotation_counts)

{'CK': 34, 'EC': 42, 'DZ': 33, 'JJ': 35, 'JK': 36}


In [18]:
for key in annotator_dfs.keys():
    annotator_dfs[key][["UEO_time","UEO_ch","10sec_ch"]] = ""
    annotator_dfs[key].to_csv(ospj(prodatapath,f"stim_seizure_annotations_{key}.csv"),index=False)
seizures_df.to_csv(ospj(prodatapath,"LF_seizure_annotations_wannotator.csv"),index=False)

## Extracting seizure annotations from iEEG

In [5]:
for i,pt in patient_table.iloc[-1:,:].iterrows():
    for ieeg_pt in pt.ieeg_ids:
        # try:
            save_path = ospj(datapath,pt.ptID)
            print(ieeg_pt,save_path)
            if not ospe(save_path):
                os.makedirs(save_path)
            wagon = Pioneer(usr,passpath,ieeg_pt)
            wagon.pull_annotations()
            wagon.filter_seizure_annotations()
            wagon.seizure_annotations.to_csv(ospj(save_path,f'seizure_annotations_{ieeg_pt}.csv'))
        # except:
        #     continue

HUP275_CCEP /mnt/leif/littlab/users/wojemann/stim-seizures/RAW_DATA/HUP275
got 100 annotations on call # 1 covering 27884764648 usec to 29467006836 usec
got 100 annotations on call # 2 covering 29497503906 usec to 30482353515 usec
got 100 annotations on call # 3 covering 30498353515 usec to 31606139648 usec
got 100 annotations on call # 4 covering 31606281250 usec to 32514449218 usec
got 80 annotations on call # 5 covering 32515988281 usec to 33463353515 usec
Filtered       8.67% of all annotations
HUP275_phaseII /mnt/leif/littlab/users/wojemann/stim-seizures/RAW_DATA/HUP275
got 100 annotations on call # 1 covering 7474268940 usec to 66969325580 usec
got 100 annotations on call # 2 covering 67122287495 usec to 109930176859 usec
got 100 annotations on call # 3 covering 109942059671 usec to 268307434592 usec
got 100 annotations on call # 4 covering 270609684592 usec to 321045575217 usec
got 100 annotations on call # 5 covering 321048633811 usec to 486457558314 usec
got 100 annotations on

### Creating stim config

In [7]:
ccep_table = pd.read_csv(ospj(datapath,'cceps_pts.csv'))

In [8]:
ccep_table.head()

,record_id,hupsubjno,ageatieegimplant,emuimplant_soz,ieeg_ictal_onset_electrodes,ieeg_ictal_onset_anat_lat,ieeg_ictal_onset_anat_spec,cceps_run1_home_meds,cceps_run1_pulsewidth,cceps_run1_sz,cceps_run1_szelecs,cceps_run1_soz_electrodes,cceps_run1_seizure_typical,cceps_run1_sz_typ_notes,cceps_hfs_done,cceps_hfs_seizure,cceps_hfs_stim_elec_sz,cceps_hfs_sz_typical
0,650,211,23.0,NaN,"LC1, LC2, LI1, LI2, LI3, LI4, LB1, LB2, LC1, L...",3.0,mesial temporal,1.0,300,0,NaN,NaN,NaN,NaN,1.0,1.0,RB1,1.0
1,651,212,55.0,NaN,NaN,NaN,NaN,1.0,300,0,NaN,NaN,NaN,NaN,1.0,1.0,LG2,2.0
2,649,213,21.0,SMA and mesial temporal,"LA1, LA2, LA3, LA4, LA5, LH3, LH4, LH5, LH6, L...",1.0,multifocal,1.0,300,0,NaN,NaN,NaN,NaN,1.0,1.0,"LJ1, LJ2, LJ3",2.0
3,652,214,40.0,NaN,"LB1, LB2, RB1, RB2",3.0,Mesial temporal,1.0,300,0,NaN,NaN,NaN,NaN,1.0,1.0,"LB1, RB1",1.0
4,658,216,21.0,NaN,"LC1, LC2, LB1, LB2",1.0,insula,1.0,300,0,NaN,NaN,NaN,NaN,1.0,1.0,LC1,1.0


### Saving stim annotations

In [5]:
for i,pt in patient_table.iloc[-1:,:].iterrows():
    for ieeg_pt in pt.ieeg_ids:
        if 'CCEP' in ieeg_pt:
            save_path = ospj(datapath,pt.ptID)
            print(ieeg_pt,save_path)
            if not ospe(save_path):
                os.makedirs(save_path)
            wagon = Pioneer(usr,passpath,ieeg_pt)
            wagon.pull_annotations()
            wagon.filter_stim_annotations()
            wagon.stim_annotations.to_csv(ospj(save_path,f'stim_annotations_{ieeg_pt}.csv'))

HUP275_CCEP /mnt/leif/littlab/users/wojemann/stim-seizures/RAW_DATA/HUP275
got 100 annotations on call # 1 covering 27884764648 usec to 29467006836 usec
got 100 annotations on call # 2 covering 29497503906 usec to 30482353515 usec
got 100 annotations on call # 3 covering 30498353515 usec to 31606139648 usec
got 100 annotations on call # 4 covering 31606281250 usec to 32514449218 usec
got 80 annotations on call # 5 covering 32515988281 usec to 33463353515 usec


In [12]:
wagon.annotations.Description.apply(lambda x: 'Closed relay' in x).sum()

0

In [6]:
wagon.stim_annotations

,Duration,Description,IEEG Time
Time,,,
1970-01-01 07:56:49.015625,0 days,"Start Stimulation from LO11 to LO12 ( F=1 Hz,...",28609.015625
1970-01-01 07:57:44.485351,0 days,"Start Stimulation from LO10 to LO11 ( F=1 Hz,...",28664.485351
1970-01-01 07:58:35.799804,0 days,"Start Stimulation from LO9 to LO10 ( F=1 Hz, ...",28715.799804
1970-01-01 07:59:21.493164,0 days,"Start Stimulation from LO8 to LO9 ( F=1 Hz, P...",28761.493164
1970-01-01 08:00:08.885742,0 days,"Start Stimulation from LO7 to LO8 ( F=1 Hz, P...",28808.885742
...,...,...,...
1970-01-01 09:10:32.478515,0 days,"Start Stimulation from LC3 to LC4 ( F=1 Hz, P...",33032.478515
1970-01-01 09:11:15.607421,0 days,"Start Stimulation from LC2 to LC3 ( F=1 Hz, P...",33075.607421
1970-01-01 09:12:00.213867,0 days,"Start Stimulation from LC1 to LC2 ( F=1 Hz, P...",33120.213867
